In [2]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import bloomfilter, network

In [2]:
# Example Bloom Filter
from helper import hash256
bit_field_size = 10
bit_field = [0] * bit_field_size
h256 = hash256(b'hello world')
bit = int.from_bytes(h256, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [3]:
# Example Bloom Filter 2
from helper import hash256
bit_field_size = 10
bit_field = [0] * bit_field_size
h = hash256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
h = hash256(b'goodbye')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


In [4]:
# Example Bloom Filter 3
from helper import hash160, hash256
bit_field_size = 10
bit_field = [0] * bit_field_size
phrase1 = b'hello world'
h1 = hash256(phrase1)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase1)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
phrase2 = b'goodbye'
h1 = hash256(phrase2)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase2)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
print(bit_field)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [5]:
# Example BIP0037 Bloom Filter
from helper import murmur3
from bloomfilter import BIP37_CONSTANT
field_size = 2
num_functions = 2
tweak = 42
bit_field_size = field_size * 8
bit_field = [0] * bit_field_size
for phrase in (b'hello world', b'goodbye'):
    for i in range(num_functions):
        seed = i * BIP37_CONSTANT + tweak
        h = murmur3(phrase, seed=seed)
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]


### Exercise 1
Given a Bloom Filter with these parameters: size=10, function count=5, tweak=99, which bits are set after adding these items?

* `b'Hello World'`
* `b'Goodbye!'`


In [6]:
# Exercise 1

from bloomfilter import BIP37_CONSTANT
from helper import murmur3, bit_field_to_bytes
field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')
# bit_field_size is 8 * field_size
bit_field_size = 8 * field_size
# create a bit field with the appropriate size
bit_field = [0] * bit_field_size
# for each item you want to add to the filter
for item in items:
    # iterate function_count number of times
    for i in range(function_count):
        # BIP0037 spec seed is i*BIP37_CONSTANT + tweak
        seed = i * BIP37_CONSTANT + tweak
        # get the murmur3 hash given that seed
        h = murmur3(item, seed=seed)
        # set the bit to be h mod the bit_field_size
        bit_field[h % bit_field_size] = 1
        # set the bit_field at the index bit to be 1

# print the bit field converted to bytes using bit_field_to_bytes in hex
print(bit_field_to_bytes(bit_field).hex())

4000600a080000010940


### Exercise 2




#### Make [this test](/edit/session8/bloomfilter.py) pass: `bloomfilter.py:BloomFilterTest:test_add`

In [7]:
# Exercise 2

reload(bloomfilter)
run(bloomfilter.BloomFilterTest('test_add'))

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


### Exercise 3




#### Make [this test](/edit/session8/bloomfilter.py) pass: `bloomfilter.py:BloomFilterTest:test_filterload`

In [8]:
# Exercise 3

reload(bloomfilter)
run(bloomfilter.BloomFilterTest('test_filterload'))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 4
Do the following:

* Connect to a testnet node
* Load a filter for your testnet address
* Send a request for transactions from the block which had your previous testnet transaction
* Receive the merkleblock and tx messages.


In [17]:
# Exercise 4
import block
import network
import script
import tx
reload(block)
reload(network)
reload(script)
reload(tx)
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from block import Block
from network import SimpleNode, GetDataMessage, BLOCK_DATA_TYPE
from tx import Tx
block_hash = bytes.fromhex('00000000000000bd8b14853cc7f1288739d557f761d97a9ac6dce3f9d57fa4dd')  # CHANGE
passphrase = b'Jimmy Song jimmy@programmingblockchain.com'  # CHANGE
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
print(addr)
filter_size = 30
filter_num_functions = 5
filter_tweak = 4747  # CHANGE
# get the hash160 of the address using decode_base58
h160 = decode_base58(addr)
# create a bloom filter using the filter_size, filter_num_functions and filter_tweak above
bf = BloomFilter(filter_size, filter_num_functions, filter_tweak)
# add the h160 to the bloom filter
bf.add(h160)
# connect to testnet.programmingbitcoin.com in testnet mode
node = SimpleNode('testnet-seed.bluematt.me', testnet=True)
# complete the handshake
node.handshake()
# create a getdata message
get_data = GetDataMessage()
# add_data (FILTERED_BLOCK_DATA_TYPE, block_hash) to request the block
get_data.add_data(BLOCK_DATA_TYPE, block_hash)
# send the getdata message
node.send(get_data)
# wait for the merkleblock command
b = node.wait_for(Block)
# check that the merkle block's hash is the same as the block hash
if b.hash() != block_hash:
    raise RuntimeError
# loop through the tx hashes we are expecting using proved_txs
for t in b.get_transactions([addr]):
    print(t)

mgHULz6rZsc9J9PbMVt4os2UGBCZV8YCHm
tx: 0d71e79c96f6f0be8622efcd7b272ee948d217df6ef5ca86c6eb98992deaf0c8
version: 1
tx_ins:
f542c6df0e0f52ab23b710ccf0cc26b376cee0618882dd65dcfef7c60c0e7450:6
tx_outs:
4000000:OP_DUP OP_HASH160 6e13971913b9aa89659a9f53d327baa8826f2d75 OP_EQUALVERIFY OP_CHECKSIG 5950000:OP_DUP OP_HASH160 086b63228be1fbeb4fceab281e19f578fa13e454 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0

tx: 0d71e79c96f6f0be8622efcd7b272ee948d217df6ef5ca86c6eb98992deaf0c8
version: 1
tx_ins:
f542c6df0e0f52ab23b710ccf0cc26b376cee0618882dd65dcfef7c60c0e7450:6
tx_outs:
4000000:OP_DUP OP_HASH160 6e13971913b9aa89659a9f53d327baa8826f2d75 OP_EQUALVERIFY OP_CHECKSIG 5950000:OP_DUP OP_HASH160 086b63228be1fbeb4fceab281e19f578fa13e454 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0



### Exercise 5




#### Make [this test](/edit/session8/network.py) pass: `network.py:SimpleNodeTest:test_get_filtered_txs`

In [15]:
# Exercise 5

reload(network)
run(network.SimpleNodeTest('test_get_filtered_txs'))

E
ERROR: test_get_filtered_txs (network.SimpleNodeTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/jimmy/pb-exercises/session8/network.py", line 484, in test_get_filtered_txs
    txs = node.get_filtered_txs([block_hash])
  File "/home/jimmy/pb-exercises/session8/network.py", line 449, in get_filtered_txs
    mb = self.wait_for(MerkleBlock)
  File "/home/jimmy/pb-exercises/session8/network.py", line 407, in wait_for
    envelope = self.read()
  File "/home/jimmy/pb-exercises/session8/network.py", line 394, in read
    envelope = NetworkEnvelope.parse(self.stream, testnet=self.testnet)
  File "/home/jimmy/pb-exercises/session8/network.py", line 47, in parse
    raise RuntimeError('Connection reset!')
RuntimeError: Connection reset!

----------------------------------------------------------------------
Ran 1 test in 0.170s

FAILED (errors=1)


### Exercise 6
You have been sent some unknown amount of testnet bitcoins to your address.

Send all of it back (minus fees) to `mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv` using only the networking protocol.

This should be a 1 input, 1 output transaction.

Remember turn on logging in `SimpleNode` if you need to debug


In [17]:
# Exercise 6

from time import sleep
from block import Block
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import decode_base58, hash160, hash256, little_endian_to_int
from merkleblock import MerkleBlock
from network import GetHeadersMessage, HeadersMessage, SimpleNode
from script import p2pkh_script
from tx import Tx, TxIn, TxOut
start_block_hex = '000000000000002020d1823678eb3094c27506c7061fb590cd83cebe051e1693'  # CHANGE
start_block = bytes.fromhex(start_block_hex)
end_block_hex = '00' * 32
end_block = bytes.fromhex(end_block_hex)
passphrase = b'Jimmy Song jimmy@programmingblockchain.com'  # CHANGE
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
h160 = decode_base58(addr)
target_address = 'mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
bloom_filter = BloomFilter(30, 5, 90210)
fee = 5000  # fee in satoshis
# connect to testnet.programmingbitcoin.com in testnet mode
node = SimpleNode('testnet-seed.bitcoin.jonasschnelli.ch', testnet=True)
# add the h160 to the bloom filter
bloom_filter.add(h160)
# complete the handshake
node.handshake()
# send the 'filterload' message from the bloom filter
node.send(bloom_filter.filterload())
# create GetHeadersMessage with the start_block as the start_block and end_block as the end block
get_headers = GetHeadersMessage(start_block=start_block, end_block=end_block)
# send a getheaders message
node.send(get_headers)
# wait for the headers message
headers = node.wait_for(HeadersMessage)
# check that the headers are valid
if not headers.is_valid():
    raise RuntimeError
# get all the block hashes from the headers.headers array
block_hashes = [header.hash() for header in headers.headers]
# get the filtered transactions from these blocks
txs = node.get_filtered_txs(block_hashes)
# loop through each filtered transaction
for tx_obj in txs:
    # use find_utxos to get utxos that belong to our address
    utxos = tx_obj.find_utxos(addr)
    # if we have any utxos, break
    if len(utxos):
        break
# prev_tx, prev_index, prev_amount are what we get in each utxo
prev_tx, prev_index, prev_amount = utxos[0]
# create tx_in
tx_in = TxIn(prev_tx, prev_index)
# calculate the output amount (prev_amount - fee)
output_amount = prev_amount - fee
# create tx_out
tx_out = TxOut(output_amount, target_script)
# create transaction on testnet
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True)
# sign the one input we have
tx_obj.sign_input(0, private_key)
# serialize and hex to see what it looks like
print(tx_obj.serialize().hex())
# send this signed transaction on the network
node.send(tx_obj)
# wait a sec so this message goes through to the other node sleep(1)
sleep(1)
# now check to see if the tx has been accepted using is_tx_accepted()
print(node.is_tx_accepted(tx_obj))



01000000017b15c56a5a81d853f96300718f6289fa4891c2dd906043742adac67f5c20344c000000006b48304502210095acf8d01af1ce04fd547dbf5d49e9913ecfd282b35068b10c2a68db182b11f6022042469d28e58e3c764c79a3790b00b6aa7749473db0517f785724f5dce0785fc401210254c7c368f901ebbc4c14479d7f90f45c886c63974727288130c110d8d778ddbbffffffff011ce9e800000000001976a9146e13971913b9aa89659a9f53d327baa8826f2d7588ac00000000
True
